In [1]:
!pip install BeautifulSoup4
!pip install lxml
!pip install requests

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
#convert the link into a txet
link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
#Creates a soup object
soup = BeautifulSoup(link, 'lxml')

In [4]:
#using soup object cretes a list ftom the "wikitable"
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

In [5]:
#convert list into Pandas DataFrame
dbtoronto = pd.DataFrame(data = data,columns = columns)
dbtoronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#Erases the rows with "Not assigned" in the Borough column
dbtoronto=dbtoronto[dbtoronto['Borough']!='Not assigned']

In [7]:
#Combines neighborhoods in the same postal code area
dbtoronto["Neighborhood"] = dbtoronto.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))
#For neighborhoods thath have Borough but "Not assigned" Neighborhood, the Borough will be also set as the Neighborhood
dbtoronto['Neighborhood'].replace("Not assigned", dbtoronto["Borough"],inplace=True)

In [8]:
#Change the index name to the postal code
dbtoronto=dbtoronto.set_index('Postal Code')

In [11]:
dbtoronto.shape


(103, 2)

In [12]:
dbtoronto


,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,Business reply mail Processing Centre
